In [1]:
import os
os.sys.path.append('./models/research/audioset/yamnet')
import params
import features as features_lib

import numpy as np
import resampy
import soundfile as sf
import tensorflow as tf

# Directory with wav files for each class
train_dir = './train_set/'
# Directory that will store the spectrogram inputs for each class
out_dir = './train_set_patches/'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    for i in os.listdir(train_dir):
        os.mkdir(out_dir+i)

FileNotFoundError: [Errno 2] No such file or directory: './train_set/'

In [2]:
for cls in os.listdir(train_dir):
    print(cls)
    for smp in os.listdir(train_dir+cls)[:10]:
        wav_data, sr = sf.read(train_dir+cls+'/'+smp, dtype=np.int16)
        assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
        input_width = round(sr*.975)
        shft = max(0, round((len(wav_data)-input_width)/2))
        wav_data = wav_data[shft:shft+input_width]
        waveform = wav_data / 32768.0
        
        if len(waveform.shape) > 1:
            waveform = np.mean(waveform, axis=1)
        if sr != params.SAMPLE_RATE:
            waveform = resampy.resample(waveform, sr, params.SAMPLE_RATE)
            
        waveform = np.reshape(waveform, [1, -1]).astype(np.float32)
        spectrogram = features_lib. \
                      waveform_to_log_mel_spectrogram(tf.squeeze(waveform, axis=0), 
                                                      params)
        patches = features_lib. \
                  spectrogram_to_patches(spectrogram, params)
        
        
        np.save(out_dir+cls+'/'+smp+'.npy', tf.squeeze(patches))

environment
chainsaw
